# UK Trafik Verisi Analizi - 2000-2016

# -------------------------------------------------------

# Kütüphanelerin yüklemmesi

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ayarları düzenle

In [2]:
plt.style.use('ggplot')
sns.set(font_scale=1.1)

# Veriyi oku

In [3]:
df = pd.read_csv("ukTrafficAADF.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'ukTrafficAADF.csv'

# 1. Eksik Veri Kontrolü

In [ ]:
missing_data = df.isnull().sum()
print("Eksik Veriler:\n", missing_data[missing_data > 0])

# 2. Yıllara Göre Toplam Trafik Hacmi

In [ ]:
yearly_traffic = df.groupby("AADFYear")["AllMotorVehicles"].sum().reset_index()
plt.figure(figsize=(12,6))
sns.lineplot(data=yearly_traffic, x="AADFYear", y="AllMotorVehicles", marker="o")
plt.title("Yıllara Göre Toplam Motorlu Araç Trafiği")
plt.xlabel("Yıl")
plt.ylabel("Toplam Motorlu Araç")
plt.tight_layout()
plt.show()


# 3. Araç Türlerinin Yıllık Değişimi

In [ ]:
vehicle_columns = ["PedalCycles", "Motorcycles", "CarsTaxis", "BusesCoaches", 
                   "LightGoodsVehicles", "AllHGVs"]

vehicle_yearly = df.groupby("AADFYear")[vehicle_columns].sum().reset_index()
plt.figure(figsize=(14,7))
for col in vehicle_columns:
    plt.plot(vehicle_yearly["AADFYear"], vehicle_yearly[col], label=col)
plt.legend()
plt.title("Araç Türlerinin Yıllara Göre Trafik Değişimi")
plt.xlabel("Yıl")
plt.ylabel("Araç Sayısı")
plt.tight_layout()
plt.show()

# 4. Bölgelere Göre Trafik Dağılımı

In [ ]:
region_traffic = df.groupby("Region")["AllMotorVehicles"].sum().sort_values(ascending=False)
plt.figure(figsize=(10,6))
sns.barplot(x=region_traffic.values, y=region_traffic.index, palette="magma")
plt.title("Bölgelere Göre Toplam Motorlu Araç Trafiği")
plt.xlabel("Toplam Araç")
plt.ylabel("Bölge")
plt.tight_layout()
plt.show()

# 5. En Yoğun İlk 10 İlçe

In [ ]:
la_traffic = df.groupby("LocalAuthority")["AllMotorVehicles"].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(10,6))
sns.barplot(x=la_traffic.values, y=la_traffic.index, palette="viridis")
plt.title("En Yoğun Trafiğe Sahip İlk 10 İlçe")
plt.xlabel("Toplam Araç")
plt.ylabel("İlçe")
plt.tight_layout()
plt.show()

# 6. Yol Kategorisine Göre Trafik

In [ ]:
road_cat = df.groupby("RoadCategory")[vehicle_columns].sum()
road_cat_percent = road_cat.div(road_cat.sum(axis=1), axis=0) * 100
road_cat_percent.plot(kind="bar", stacked=True, figsize=(12,6), colormap="tab10")
plt.title("Yol Kategorilerine Göre Araç Dağılım Yüzdeleri")
plt.ylabel("Yüzde (%)")
plt.xlabel("Yol Kategorisi")
plt.legend(title="Araç Türü")
plt.tight_layout()
plt.show()

# 7. Yoğunluk Analizi (Araç / km)

In [ ]:
df["TrafficDensity"] = df["AllMotorVehicles"] / df["LinkLength_km"]
top_density = df.sort_values(by="TrafficDensity", ascending=False).head(10)

plt.figure(figsize=(12,6))
sns.barplot(x=top_density["TrafficDensity"], y=top_density["Road"], palette="rocket")
plt.title("En Yoğun Trafikli 10 Yol (Araç / km)")
plt.xlabel("Araç Yoğunluğu (Araç / km)")
plt.ylabel("Yol")
plt.tight_layout()
plt.show()

# 8. Kırsal vs Kentsel (Yol Türüne Göre)

In [ ]:
rural_urban_map = {
    "A": "Urban", "B": "Rural", "C": "Rural", "M": "Urban", "TM": "Urban"
}

df["AreaType"] = df["RoadCategory"].map(lambda x: rural_urban_map.get(x[0], "Unknown"))

area_traffic = df.groupby("AreaType")["AllMotorVehicles"].sum()
plt.figure(figsize=(7,5))
area_traffic.plot(kind="pie", autopct='%1.1f%%', startangle=90, shadow=True, colors=["#66c2a5", "#fc8d62"])
plt.ylabel("")
plt.title("Kentsel ve Kırsal Alanlara Göre Trafik Dağılımı")
plt.tight_layout()
plt.show()